<a href="https://colab.research.google.com/github/Amanda0975/MT/blob/main/HHI_Riqueza_2021_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [25]:
import requests
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

# Enlace del archivo en Dropbox
url = 'https://www.dropbox.com/scl/fi/ecy7lnydd0zszwvvmp8o2/data.csv.zip?dl=0&e=1&rlkey=zqyvspuxoqyxvrkr6bkjwtjik'

# Ajustar el enlace para descargar el archivo directamente
url = url.replace("?dl=0", "?dl=1")

# Descarga el archivo ZIP desde el enlace de Dropbox
response = requests.get(url)
response.raise_for_status()  # Verifica si la descarga fue exitosa

# Lee el archivo ZIP en memoria
zip_file = ZipFile(BytesIO(response.content))

# Extrae y lee el archivo CSV del archivo ZIP
with zip_file.open(zip_file.namelist()[0]) as csv_file:
    df = pd.read_csv(csv_file)

<ipython-input-25-01568c2e2492>:21: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [26]:
data=df.copy()

In [27]:
#Creación columna "Region_id"
# Definir un diccionario de mapeo para los valores de la columna "Región" a "region_id"
region_mapping = {
    'XV REGION ARICA Y PARINACOTA': 15,
    'I REGION DE TARAPACA': 1,
    'II REGION DE ANTOFAGASTA': 2,
    'III REGION DE ATACAMA': 3,
    'IV REGION COQUIMBO': 4,
    'V REGION VALPARAISO': 5,
    'XIII REGION METROPOLITANA': 13,
    'VI REGION DEL LIBERTADOR GENERAL BERNARDO O\'HIGGINS': 6,
    'VII REGION DEL MAULE': 7,
    'XVI REGION DE ÑUBLE': 16,
    'VIII REGION DEL BIO BIO': 8,
    'IX REGION DE LA ARAUCANIA': 9,
    'XIV REGION DE LOS RIOS': 14,
    'X REGION LOS LAGOS': 10,
    'XI REGION AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO': 11,
    'XII REGION DE MAGALLANES Y LA ANTARTICA CHILENA': 12
}

# Aplicar el mapeo a la columna "Región" para crear la nueva columna "region_id" en el DataFrame "df"
data['region_id'] = data['Región'].map(region_mapping)

In [28]:
#Creación columna "ventas" punto medio
ventas_pm = {
    1: 0,
    2: 100,
    3: 400,
    4: 1500,
    5: 3700,
    6: 7500,
    7: 17500,
    8: 37500,
    9: 75000,
    10: 150000,
    11: 400000,
    12: 800000,
    13: 1000000
}

data['aporte'] = data['Tramo según ventas'].map(ventas_pm)

In [29]:
#Primer filtro
data = data[data['Año comercial'].isin([2021, 2022])].drop_duplicates()
data = data[data['Tipo de contribuyente'].isin(['PERSONA JURIDICA COMERCIAL'])].drop_duplicates()

In [30]:
#Agrupar por comuna, año comercial y rubro
data_1=data[['region_id', 'Comuna']].copy()
data = data.groupby(['Comuna', 'Año comercial', 'Rubro económico']).sum().reset_index().drop_duplicates()
data = data.drop(columns=['region_id'])
data=pd.merge(data,data_1, on="Comuna")

In [33]:
#Eliminar columnas que contienen los nombres
eliminar = ["DV", "Razón social", "Número de trabajadores dependie", "Fecha inicio de actividades vige","Fecha término de giro","Fecha primera inscripción de ac","Tipo término de giro","Subtipo de contribuyente","Tramo capital propio positivo","Tramo capital propio negativo","Subrubro económico","Actividad económica","Provincia","R_PRESUNTA","OTROS_REGIMENES","RUT","Región", "Tramo según ventas", "Tipo de contribuyente" ]
columnas = [col for col in df.columns if any(word in col for word in eliminar)]

# Eliminar columnas seleccionadas
data = data.drop(columns=columnas).drop_duplicates()

In [34]:
#Separación por año
df1 = data[data['Año comercial'].isin([2021])].drop_duplicates()
df2 = data[data['Año comercial'].isin([2022])].drop_duplicates()

##Año 2021 (df1)

In [35]:
df1 = df1.drop(columns=['Año comercial', 'region_id'])
df_pivot = df1.pivot_table(index='Comuna', columns='Rubro económico', values='aporte', aggfunc='sum', fill_value=0)
df1 = pd.merge(df1, df_pivot, on="Comuna")
df1 = df1.drop(columns=['Rubro económico']).drop_duplicates()

In [38]:
#Agrupar aporte por comuna
df1_1 = df1[['aporte', 'Comuna']].copy()
df1_1 = df1_1.groupby('Comuna', as_index=False)['aporte'].sum().drop_duplicates()

#Eliminar aporte
df1=df1.drop(columns=['aporte']).drop_duplicates()

#Juntar ambos df
df1 = pd.merge(df1, df1_1, on="Comuna")

In [39]:
rubro = ['ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS','ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS',
         'ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL','ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES',
         'ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES','ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO',
         'ACTIVIDADES FINANCIERAS Y DE SEGUROS','ACTIVIDADES INMOBILIARIAS', 'ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS',
         'ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA','AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA',
         'COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS', 'CONSTRUCCION','ENSEÑANZA','EXPLOTACION DE MINAS Y CANTERAS','INDUSTRIA MANUFACTURERA',
         'INFORMACION Y COMUNICACIONES', 'OTRAS ACTIVIDADES DE SERVICIOS','SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION',
         'SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO','TRANSPORTE Y ALMACENAMIENTO','Valor por Defecto']

df1_copy = df1.copy()

# Divide cada columna en "rubros" por la columna "aporte"
df1_copy[rubro] = df1_copy[rubro].div(df1_copy['aporte'], axis=0)

df1=df1_copy

In [40]:
#Merge
data1=data[['region_id', 'Comuna']].copy()
df1=pd.merge(df1, data1, on="Comuna").drop_duplicates()

In [44]:
df1_1=df1.copy()

In [45]:
for column in rubro:
    if column in df1_1.columns:
        df1_1[column] = df1_1[column] ** 2

In [51]:
df1_1['HHI_2021'] = df1_1[[col for col in rubro if col in df1_1.columns]].sum(axis=1)
df1=df1_1.copy()

In [66]:
display(df1)

,Comuna,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS",ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL,ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES,ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO,ACTIVIDADES FINANCIERAS Y DE SEGUROS,ACTIVIDADES INMOBILIARIAS,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",...,INDUSTRIA MANUFACTURERA,INFORMACION Y COMUNICACIONES,OTRAS ACTIVIDADES DE SERVICIOS,"SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION","SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",TRANSPORTE Y ALMACENAMIENTO,Valor por Defecto,aporte,region_id,HHI_2021
0,ALGARROBO,3.840491e-09,0.002460,5.753731e-05,0.0,0.0,1.460347e-04,1.589625e-01,0.025130,0.000593,...,3.080159e-04,2.636266e-06,9.831656e-07,3.097586e-06,1.009392e-02,0.000007,0.000000e+00,8068200,5.0,0.232243
37,ALHUE,0.000000e+00,0.000407,1.652993e-05,0.0,0.0,2.525271e-03,1.719774e-03,0.000090,0.000035,...,3.947439e-03,6.611972e-07,0.000000e+00,0.000000e+00,7.522955e-07,0.002078,0.000000e+00,1844700,13.0,0.365043
69,ALTO BIO BIO,0.000000e+00,0.003645,9.585063e-06,0.0,0.0,2.396266e-06,1.834641e-02,0.042069,0.000000,...,0.000000e+00,1.347899e-04,0.000000e+00,0.000000e+00,3.369749e-03,0.000776,0.000000e+00,129200,8.0,0.286206
96,ALTO DEL CARMEN,5.374911e-07,0.000000,0.000000e+00,0.0,0.0,1.209355e-04,0.000000e+00,0.003023,0.000000,...,1.770904e-01,8.599857e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.003105,0.000000e+00,136400,3.0,0.354667
115,ALTO HOSPICIO,4.225004e-08,0.000024,1.888153e-05,0.0,0.0,3.711791e-03,5.299846e-06,0.000592,0.000025,...,5.246258e-03,9.229455e-06,4.208121e-06,3.992408e-05,4.928045e-08,0.027574,2.440363e-08,24325200,1.0,0.282337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11556,VITACURA,1.114112e-05,0.000644,9.963575e-04,0.0,0.0,8.175812e-04,4.487391e-02,0.020642,0.005083,...,1.604604e-03,4.635693e-04,4.975222e-05,1.215290e-05,8.132923e-05,0.000356,2.913890e-09,424227600,13.0,0.149019
11597,YERBAS BUENAS,0.000000e+00,0.000563,5.334262e-08,0.0,0.0,2.700470e-07,5.334262e-08,0.000015,0.000022,...,1.205416e-01,4.814172e-06,9.635011e-07,1.875327e-05,0.000000e+00,0.000622,0.000000e+00,1731900,7.0,0.268537
11630,YUMBEL,0.000000e+00,0.000256,7.624115e-07,0.0,0.0,5.557980e-04,4.758210e-03,0.000030,0.000004,...,3.889036e-03,4.088722e-06,6.842696e-07,2.891250e-06,0.000000e+00,0.001295,0.000000e+00,2176000,8.0,0.502537
11664,YUNGAY,7.251198e-08,0.000192,2.273976e-06,0.0,0.0,8.680844e-05,3.139508e-04,0.000168,0.000038,...,5.169306e-03,1.160192e-06,1.403832e-06,0.000000e+00,0.000000e+00,0.002267,0.000000e+00,1856800,16.0,0.334529


###Riqueza 2021

In [67]:
riqueza_2021=df1[['region_id','Comuna','aporte']].copy()
riqueza_2021=riqueza_2021.rename(columns={'aporte': 'riqueza_2021'})
display(riqueza_2021)

,region_id,Comuna,riqueza_2021
0,5.0,ALGARROBO,8068200
37,13.0,ALHUE,1844700
69,8.0,ALTO BIO BIO,129200
96,3.0,ALTO DEL CARMEN,136400
115,1.0,ALTO HOSPICIO,24325200
...,...,...,...
11556,13.0,VITACURA,424227600
11597,7.0,YERBAS BUENAS,1731900
11630,8.0,YUMBEL,2176000
11664,16.0,YUNGAY,1856800


##Año 2022 (df2)

In [57]:
df2 = df2.drop(columns=['Año comercial', 'region_id'])
df_pivot = df2.pivot_table(index='Comuna', columns='Rubro económico', values='aporte', aggfunc='sum', fill_value=0)
df2 = pd.merge(df2, df_pivot, on="Comuna")
df2 = df2.drop(columns=['Rubro económico']).drop_duplicates()

In [58]:
#Agrupar aporte por comuna
df2_1 = df2[['aporte', 'Comuna']].copy()
df2_1 = df2_1.groupby('Comuna', as_index=False)['aporte'].sum().drop_duplicates()

#Eliminar aporte
df2=df2.drop(columns=['aporte']).drop_duplicates()

#Juntar ambos df
df2 = pd.merge(df2, df2_1, on="Comuna")

In [59]:
rubro = ['ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS','ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS',
         'ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL','ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES',
         'ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES','ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO',
         'ACTIVIDADES FINANCIERAS Y DE SEGUROS','ACTIVIDADES INMOBILIARIAS', 'ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS',
         'ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA','AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA',
         'COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS', 'CONSTRUCCION','ENSEÑANZA','EXPLOTACION DE MINAS Y CANTERAS','INDUSTRIA MANUFACTURERA',
         'INFORMACION Y COMUNICACIONES', 'OTRAS ACTIVIDADES DE SERVICIOS','SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION',
         'SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO','TRANSPORTE Y ALMACENAMIENTO','Valor por Defecto']

df2_copy = df2.copy()

# Divide cada columna en "rubros" por la columna "aporte"
df2_copy[rubro] = df2_copy[rubro].div(df2_copy['aporte'], axis=0)

df2=df2_copy

In [60]:
#Merge
data2=data[['region_id', 'Comuna']].copy()
df2=pd.merge(df2, data2, on="Comuna").drop_duplicates()

In [61]:
for column in rubro:
    if column in df1.columns:
        df2[column] = df2[column] ** 2

In [64]:
df2['HHI_2022'] = df2[[col for col in rubro if col in df2.columns]].sum(axis=1)

In [65]:
display(df2)

,Comuna,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS",ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL,ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES,ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO,ACTIVIDADES FINANCIERAS Y DE SEGUROS,ACTIVIDADES INMOBILIARIAS,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",...,INFORMACION Y COMUNICACIONES,OTRAS ACTIVIDADES DE SERVICIOS,"SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION","SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",TRANSPORTE Y ALMACENAMIENTO,Valor por Defecto,aporte,region_id,HHI_2021,HHI_2022
0,ALGARROBO,2.393722e-07,0.002522,2.798504e-05,0.0,0.0,0.000722,7.596342e-02,0.074894,0.000183,...,3.403677e-06,7.157269e-07,1.059824e-06,7.764637e-03,0.000041,0.000000e+00,9810800,5.0,0.191750,0.191750
37,ALHUE,0.000000e+00,0.000810,3.776723e-06,0.0,0.0,0.004843,3.900205e-04,0.000000,0.000014,...,0.000000e+00,1.510689e-05,0.000000e+00,2.758746e-09,0.000317,0.000000e+00,1903900,13.0,0.354399,0.354399
69,ALTO BIO BIO,0.000000e+00,0.001986,6.645794e-05,0.0,0.0,0.000007,2.392486e-03,0.050137,0.000066,...,4.725898e-06,0.000000e+00,0.000000e+00,9.045664e-03,0.011463,0.000000e+00,184000,8.0,0.268079,0.268079
96,ALTO DEL CARMEN,3.925855e-07,0.000000,0.000000e+00,0.0,0.0,0.000101,0.000000e+00,0.000567,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.002268,0.000000e+00,159600,3.0,0.301892,0.301892
115,ALTO HOSPICIO,9.177744e-07,0.000032,6.464583e-06,0.0,0.0,0.003552,8.470406e-06,0.000150,0.000487,...,7.014856e-05,7.801809e-06,2.191070e-05,1.474641e-07,0.022529,6.535807e-11,24738900,1.0,0.231088,0.231088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11556,VITACURA,4.322485e-05,0.000890,1.100192e-03,0.0,0.0,0.001449,4.502105e-02,0.018029,0.004884,...,6.979408e-04,3.700995e-05,1.853258e-08,1.399436e-04,0.000330,2.506278e-08,423845800,13.0,0.144940,0.144940
11597,YERBAS BUENAS,2.473463e-09,0.001486,5.565292e-07,0.0,0.0,0.001745,5.565292e-07,0.000051,0.000007,...,1.354468e-05,9.893853e-07,1.683501e-03,0.000000e+00,0.001074,0.000000e+00,2010700,7.0,0.223434,0.223434
11630,YUMBEL,8.419890e-09,0.000210,1.091218e-05,0.0,0.0,0.000830,1.184047e-05,0.000245,0.000007,...,2.728044e-06,8.419890e-09,0.000000e+00,0.000000e+00,0.000979,0.000000e+00,2179600,8.0,0.270212,0.270212
11664,YUNGAY,3.319048e-08,0.000134,3.189605e-05,0.0,0.0,0.000046,3.487075e-06,0.000038,0.000001,...,6.721072e-05,5.995031e-07,6.721072e-07,0.000000e+00,0.001460,0.000000e+00,2195600,16.0,0.380910,0.380910


###Riqueza 2022

In [68]:
riqueza_2022=df2[['region_id','Comuna','aporte']].copy()
riqueza_2022=riqueza_2022.rename(columns={'aporte': 'riqueza_2022'})
display(riqueza_2022)

,region_id,Comuna,riqueza_2022
0,5.0,ALGARROBO,9810800
37,13.0,ALHUE,1903900
69,8.0,ALTO BIO BIO,184000
96,3.0,ALTO DEL CARMEN,159600
115,1.0,ALTO HOSPICIO,24738900
...,...,...,...
11556,13.0,VITACURA,423845800
11597,7.0,YERBAS BUENAS,2010700
11630,8.0,YUMBEL,2179600
11664,16.0,YUNGAY,2195600


#Exportar

In [69]:
df1.to_csv('HHI_2021.csv', index=False)
df2.to_csv('HHI_2022.csv', index=False)
riqueza_2021.to_csv('riqueza_2021.csv', index=False)
riqueza_2022.to_csv('riqueza_2022.csv', index=False)